In [1]:
%%writefile serve.py

import os
import joblib
import pandas as pd

def model_fn(model_dir):
    """Load and return the model"""
    model_file_name="pipeline_model.joblib"
    pipeline_model=joblib.load(os.path.join(model_dir,model_file_name))

    return pipeline_model

def input_fn(request_body, request_content_type):
    """Process the input jason data and return the processed data.
    You can also add any input data pre-processing in this function"""
    if request_content_type=="application/json":
        input_object=pd.read_json(request_body, lines=True)
        return input_object
    else:
        raise ValueError("Only application/json content type supported!")

def predict_fn(input_object, pipeline_model):
    """Make predictions on processed input data"""
    predictions=pipeline_model.predict(input_object)
    pred_probs=pipeline_model.predict_proba(input_object)

    prediction_object=pd.DataFrame(
        {
            "prediction":predictions.tolist(),
            "pred_prob_class0":pred_probs[:,0].tolist(),
            "pred_prob_class1":pred_probs[:,1].tolist()
        }
    )
    return prediction_object

def output_fn(prediction_object, request_content_type):
    """Post processthe predictionds and return as json"""
    return_object= prediction_object.to_json(orient="records", lines=True)

    return return_object
    
        

Overwriting serve.py


In [2]:
%%writefile requirements.txt
pandas
numpy

Overwriting requirements.txt


In [3]:
# Create the deployment - same as Real Time Inference Code!
from sagemaker.sklearn.model import SKLearnModel
from sagemaker import Session, get_execution_role

session= Session()
bucket= session.default_bucket()

training_job_name="pipeline-run-240303-0855-001-734d53ae" #TODO; Update with best TrainingJobName from hyperparameter tuning
model_artifact= f"s3://{bucket}/{training_job_name}/output/model.tar.gz"
endpoint_name= "pipeline-batch-transform"

model=SKLearnModel(
    framework_version="1.0-1",
    entry_point="serve.py",
    dependencies=["requirements.txt"],
    model_data= model_artifact,
    role=get_execution_role(),
    sagemaker_session=session,
    name = 'salary-prediction-pipeline-batch-transform'
)

                  

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [4]:
# NEW! Create a batch transformer from the base model
output_path= f"s3://{bucket}/sagemaker/salary_preds"
batch_transformer=model.transformer(instance_count=2,
                                    instance_type="ml.m5.large",
                                    strategy="MultiRecord",
                                    accept="application/json",
                                    assemble_with="Line",
                                    output_path=output_path)

Using already existing model: salary-prediction-pipeline-batch-transform


In [5]:
%%time
# NEW! Feed the test data
test_data_path= 's3://sagemaker-us-east-1-635439539142/akash/salary-prediction/test_data_salary_pred_json.json'
batch_transformer.transform(test_data_path, split_type="Line")

INFO:sagemaker:Creating transform job with name: salary-prediction-pipeline-batch-transf-2024-03-03-11-33-50-595


ResourceLimitExceeded: An error occurred (ResourceLimitExceeded) when calling the CreateTransformJob operation: The account-level service limit 'ml.m5.large for transform job usage' is 0 Instances, with current utilization of 0 Instances and a request delta of 2 Instances. Please use AWS Service Quotas to request an increase for this quota. If AWS Service Quotas is not available, contact AWS support to request an increase for this quota.

In [ ]:
# Print the output path
output-path= f"{batch_transformer.output_path}/test_data_salary_pred_json.json.out"
print("Output written to: ")
print(f"{output_path}")

In [ ]:
import pandas as pd 
preds_df=pd.read_json(output_path, lines=True)
print(preds_df.shape)
preds_df.head()

In [6]:
# Join the predictions to input
import pandas as pd
import numpy as np
bigtest="../data/test_data_salary_pred_json.json"
bigtest_df=pd.read_json(bigtest,lines=True)
#bigtest_df= bigtest_df.join(preds_df)
bigtest_df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,0
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,0
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,1
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,1
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,0


In [ ]:
# Calculate the test accuracy
len(bigtest_df[bigtest_df["salary"]==bigtest_df["prediction"]])/len(bigtest_df)

In [7]:
import boto3

sm_client=boto3.client("sagemaker")
sm_client.delete_model(ModelName='salary-prediction-pipeline-batch-transform')

{'ResponseMetadata': {'RequestId': 'dff8e1a4-009f-4fc3-9c8e-8e216acf2858',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'dff8e1a4-009f-4fc3-9c8e-8e216acf2858',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Sun, 03 Mar 2024 11:36:50 GMT'},
  'RetryAttempts': 0}}